In [2]:
from datasets import load_dataset

dataset = load_dataset("squad")
print(dataset["train"][0])  # First training example

/Users/nirajanpaudel17/miniconda3/envs/langchain-rag/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}


In [3]:
dataset.save_to_disk("./squad_dataset")

Saving the dataset (1/1 shards): 100%|██████████| 10570/10570 [00:00<00:00, 246848.78 examples/s]


In [ ]:
from datasets import load_from_disk

dataset = load_from_disk("/Users/nirajanpaudel17/Documents/Projects/Huggingface-dataset_RAG/squad_dataset")  
train_dataset = dataset["train"] 

In [12]:
train_dataset

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [14]:
len(train_dataset)

87599

In [118]:
train_dataset[300]

{'id': '56d43da72ccc5a1400d830c1',
 'title': 'Beyoncé',
 'context': 'Following the disbandment of Destiny\'s Child in June 2005, she released her second solo album, B\'Day (2006), which contained hits "Déjà Vu", "Irreplaceable", and "Beautiful Liar". Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her marriage to rapper Jay Z and portrayal of Etta James in Cadillac Records (2008) influenced her third album, I Am... Sasha Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and earned a record-setting six Grammy Awards in 2010, including Song of the Year for "Single Ladies (Put a Ring on It)". Beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. Her critically acclaimed fifth studio album, Beyoncé (2013), was distinguished 

In [18]:
page_content = [item['context'] for item in train_dataset]

In [22]:
len(page_content), len(page_content[0])

(87599, 695)

In [ ]:
from langchain.docstore.document import Document

docs = [Document(page_content=item["context"], metadata={"title": item["title"]}) for item in train_dataset]

In [34]:
type(docs[0]), len(docs)

(langchain_core.documents.base.Document, 87599)

In [38]:
len(docs[0].page_content)

695

Pinecone database

In [ ]:
api_key ="pcsk_62ZRRY_GrCLAQuYZVRHoWXgykNVD7ZSXqyvSHD1fAEXYqhPxZxC1WVushAnCKHVXDJywgu"


pinecone needs embeddings size

In [72]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="llama3.2",
)

In [74]:
len(embeddings.embed_query("hello"))

3072

In [87]:
from pinecone import Pinecone, ServerlessSpec

index_name = "langchain-test"

pc = Pinecone(api_key=api_key)

In [93]:
index = pc.Index(index_name)

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(
    index_name=index_name,
    embedding=embeddings,
    pinecone_api_key=api_key,
)

In [123]:
# docs[:10]

In [99]:
vector_store.add_documents(docs[:10])

['92063409-9dc6-44ec-9510-ab43ad897095',
 '6cb1f402-ecc4-4fae-9e23-c7547ad5f84f',
 '5a7c30cf-f855-4abd-b48d-5294ab82f7fb',
 '5fe635d7-b254-48e6-b407-776123522ca1',
 '94ee775f-9c52-43bf-bcb7-19247b2188c5',
 '3dd8a505-6445-495a-aa3f-016b2f286127',
 '690111d6-f454-48d8-9b5c-ba75e8c46fc4',
 '6441f73e-df61-474a-89d6-42f4612e1351',
 'af3ad6e7-5f3d-4153-86ad-3591e517890a',
 '53a9e1f3-f434-42fb-978f-80312d80b34e']

In [121]:
vector_store.similarity_search("notre dam", k=1)

[Document(id='3dd8a505-6445-495a-aa3f-016b2f286127', metadata={'title': 'University_of_Notre_Dame'}, page_content="As at most other universities, Notre Dame's students run a number of news media outlets. The nine student-run outlets include three newspapers, both a radio and television station, and several magazines and journals. Begun as a one-page journal in September 1876, the Scholastic magazine is issued twice monthly and claims to be the oldest continuous collegiate publication in the United States. The other magazine, The Juggler, is released twice a year and focuses on student literature and artwork. The Dome yearbook is published annually. The newspapers have varying publication interests, with The Observer published daily and mainly reporting university and other news, and staffed by students from both Notre Dame and Saint Mary's College. Unlike Scholastic and The Dome, The Observer is an independent publication and does not have a faculty advisor or any editorial oversight f